In [48]:
import pandas as pd
import s3fs
import folium


In [49]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [50]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [51]:
df = pd.read_csv('post_processed_listings.csv')

In [52]:
df.head()

,id,Type,Nb_chambre,loc,surface,prix
0,apimo-85473173,Appartement,1,Châtelet,29,364000
1,hektor-Les-4-Quartiers-383,Appartement,1,Châtelet,23,301000
2,hektor-PARISLUXURYHOMES-4007,Appartement,5,Palais Royal,106,2490000
3,ag754594-448898324,Studio,1,Saint-Germain,27,365000
4,keller-williams-1-34_1_53-180645,Appartement,2,Vendôme,67,1190000


In [53]:
# Extraire la colonne contenant les informations sur les quartiers ou adresses
# En me basant sur le nom de la colonne, je vais essayer de cibler la bonne colonne
df_quartiers = df.iloc[:, 3]
# Afficher les premières valeurs de cette colonne pour vérifier
df_quartiers.head()

0         Châtelet
1         Châtelet
2     Palais Royal
3    Saint-Germain
4          Vendôme
Name: loc, dtype: object

In [54]:
### EXTRAIRE LES COORDONNÉES GRAPHIQUES D'UNE LOCALISATION ###
import geopy
from geopy.geocoders import Nominatim
# Création d'un objet géocodeur Nominatim
geolocator = Nominatim(user_agent="my_geocoder")
# Géocodage d'une adresse
location = geolocator.geocode("Paris Palais Royal")
# Affichage des informations de localisation
# Affichage des informations de localisation
if location:
    print("Adresse complète :", location.address)
    print("Latitude :", location.latitude)
    print("Longitude :", location.longitude)
else:
    print("Adresse introuvable.")

Adresse complète : Palais Royal, Rue de Montpensier, Quartier du Palais Royal, Paris 1er Arrondissement, Paris, Île-de-France, France métropolitaine, 75001, France
Latitude : 48.863584700000004
Longitude : 2.3362042200938715


In [55]:
### Créer une liste unique des quartiers/adresses ###. --> prendre la liste clean que Maxime fait 
# Extraire la colonne contenant les quartiers/adresses
df_quartiers = df.iloc[:, 3]
# Créer une liste des adresses uniques
list_quartiers_uniques = df_quartiers.unique()
# Afficher les premières adresses uniques pour vérification
print(list_quartiers_uniques)  # Affiche les 10 premières adresses uniques
len(list_quartiers_uniques)



['Châtelet' 'Palais Royal' 'Saint-Germain' 'Vendôme' 'Gaillon'
 "Saint-Thomas d'Aquin" '75001 Paris 1er' 'Sentier' 'Montorgueil'
 '75002 Paris 2e' 'Arts-et-Métiers' 'Sainte-Avoye' 'Archives'
 '75003 Paris 3e' 'Enfants Rouges' 'République' 'Saint-Merri' 'Arsenal'
 '75004 Paris 4e' 'Les Iles' 'Saint-Gervais' '75000 Paris' 'Monge'
 'Jardin des Plantes' 'Val de Grâce' 'Sorbonne' 'Rennes' 'Montparnasse'
 'Saint-Germain des Près' 'Odéon' 'Saint-Placide' 'Notre Dame des Champs'
 '75006 Paris 6e' 'Saint-Michel' 'Hoche Friedland' 'Elysée' 'Monceau'
 "Triangle d'Or" 'Europe' 'Saint-Philippe du Roule' 'Mairie'
 '75008 Paris 8e' 'Clichy' 'Grands Boulevards' 'Lorette' 'Opéra'
 'Trudaine' 'Porte Saint-Martin' '75009 Paris 9e' 'Saint-Vincent de Paul'
 "Château d'Eau" 'Goncourt' 'Canal Saint-Martin' 'Louis Blanc'
 'Belleville' '75010 Paris 10e' 'Nation' 'Léon-Blum' '75011 Paris 11e'
 'Bastille' '75012 Paris 12e' 'Bel-Air' 'Aligre' 'Vallée de Fecamp'
 'Jardin de Reuilly' 'Bercy' 'Olympiades' 'Croulebar

140

In [56]:
### PRENDRE *TOUTES* LES COORDONNÉES GRAPHIQUES ### --> pour l'instant j'en rentre 3 à la mains pour éviter les problèmes
from geopy.geocoders import Nominatim
import time

# Créer un objet géocodeur Nominatim
geolocator = Nominatim(user_agent="my_geocoder")

# Fonction pour obtenir les coordonnées géographiques d'une adresse
def obtenir_coordonnees(adresse):
    try:
        location = geolocator.geocode(adresse, timeout=5)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Erreur pour {adresse}: {e}")
        return None, None

# Extraire seulement les premières adresses uniques
list_quartiers_uniques_3 = ['75003 Paris 3e', '75013 Paris 13e', '75010 Paris 10e'] # ENTRER À LA MAIN EN ATTENDANT
# Créer un dictionnaire pour stocker les adresses et leurs coordonnées
coordonnees_dict = {}

# Obtenir les coordonnées pour les 10 premières adresses
for adresse in list_quartiers_uniques_3:
    lat, lon = obtenir_coordonnees(adresse)
    coordonnees_dict[adresse] = (lat, lon)
    
    # Ajouter une pause pour éviter un blocage par le serveur
    time.sleep(1)  # Pause de 1 seconde entre les requêtes

# Afficher les coordonnées pour les 3 adresses
print("\nCoordonnées des 3 premières adresses :")
for adresse, coord in coordonnees_dict.items():
    print(f"Adresse: {adresse}, Latitude: {coord[0]}, Longitude: {coord[1]}")



Coordonnées des 3 premières adresses :
Adresse: 75003 Paris 3e, Latitude: 48.864212, Longitude: 2.360936
Adresse: 75013 Paris 13e, Latitude: 48.8290105, Longitude: 2.3636124
Adresse: 75010 Paris 10e, Latitude: 48.876225, Longitude: 2.3595209


In [57]:
### CREATION DE LA CARTE ###

# Coordonnées des adresses
coordonnees = [
    {"adresse": "75003 Paris 3e", "latitude": 48.864212, "longitude": 2.360936},
    {"adresse": "75013 Paris 13e", "latitude": 48.8290105, "longitude": 2.3636124},
    {"adresse": "75010 Paris 10e", "latitude": 48.876225, "longitude": 2.3595209},
]

# Initialiser la carte centrée sur Paris
carte = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

# Ajouter des points sur la carte pour les adresses avec coordonnées
for lieu in coordonnees:
    if lieu["latitude"] is not None and lieu["longitude"] is not None:
        folium.Marker(
            location=[lieu["latitude"], lieu["longitude"]],
            popup=lieu["adresse"],  # Texte affiché lorsqu'on clique sur le point
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(carte)

# Sauvegarder la carte dans un fichier HTML
carte.save("carte_paris_points.html")
print("Carte créée : carte_paris_points.html")



Carte créée : carte_paris_points.html


In [58]:
carte

In [59]:
## CODE MAXIME ##

In [60]:
# Uniformiser le format des localisation
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 10)
df_ = df.copy()
df_[['parentheses']] = df_['loc'].str.extract(r'\((.*?)\)')
df_['concat'] = df_['parentheses'].fillna('')
df_['loc'] = df_['concat'].where(df_['parentheses'].notna(), df_['loc'])
df_.drop(columns=['parentheses', 'concat'], inplace=True)

df_[['parentheses']] = df_['loc'].str.extract(r'^(.*?)\s*\s-')
df_['concat'] = df_['parentheses'].fillna('')
df_['loc'] = df_['concat'].where(df_['parentheses'].notna(), df_['loc'])
df_.drop(columns=['parentheses', 'concat'], inplace=True)

df_zones=df_["loc"].drop_duplicates().copy()
df=df_.copy()

In [61]:
from geopy.geocoders import Nominatim
from functools import partial
geolocator = Nominatim(user_agent="testing")
geocode = partial(geolocator.geocode, language="fr")
geocode_with_paris = lambda query: geolocator.geocode("%s, Paris FR" % query)

In [62]:
# Initialiser une liste vide pour stocker les zones
tableau_zones = []

# Parcourir les zones et les ajouter au tableau
for zone in df_zones:
    print(zone)  # Affiche chaque zone
    tableau_zones.append(zone)  # Ajoute chaque zone au tableau

# Afficher le tableau final après la boucle
print("\nTableau des zones :")
print(tableau_zones)

Châtelet
Palais Royal
Saint-Germain
Vendôme
Gaillon
Saint-Thomas d'Aquin
75001 Paris 1er
Sentier
Montorgueil
75002 Paris 2e
Arts-et-Métiers
Sainte-Avoye
Archives
75003 Paris 3e
Enfants Rouges
République
Saint-Merri
Arsenal
75004 Paris 4e
Les Iles
Saint-Gervais
75000 Paris
Monge
Jardin des Plantes
Val de Grâce
Sorbonne
Rennes
Montparnasse
Saint-Germain des Près
Odéon
Saint-Placide
Notre Dame des Champs
75006 Paris 6e
Saint-Michel
Hoche Friedland
Elysée
Monceau
Triangle d'Or
Europe
Saint-Philippe du Roule
Mairie
75008 Paris 8e
Clichy
Grands Boulevards
Lorette
Opéra
Trudaine
Porte Saint-Martin
75009 Paris 9e
Saint-Vincent de Paul
Château d'Eau
Goncourt
Canal Saint-Martin
Louis Blanc
Belleville
75010 Paris 10e
Nation
Léon-Blum
75011 Paris 11e
Bastille
75012 Paris 12e
Bel-Air
Aligre
Vallée de Fecamp
Jardin de Reuilly
Bercy
Olympiades
Croulebarbe
Bièvre Sud
Patay
Nationale
Salpétrière
Butte-aux-Cailles
Bibliothèque
75013 Paris 13e
94200 Ivry-sur-Seine
Pernety
Jean Moulin
Didot
Montsouris
750

In [63]:
tableau_zones

['Châtelet',
 'Palais Royal',
 'Saint-Germain',
 'Vendôme',
 'Gaillon',
 "Saint-Thomas d'Aquin",
 '75001 Paris 1er',
 'Sentier',
 'Montorgueil',
 '75002 Paris 2e',
 'Arts-et-Métiers',
 'Sainte-Avoye',
 'Archives',
 '75003 Paris 3e',
 'Enfants Rouges',
 'République',
 'Saint-Merri',
 'Arsenal',
 '75004 Paris 4e',
 'Les Iles',
 'Saint-Gervais',
 '75000 Paris',
 'Monge',
 'Jardin des Plantes',
 'Val de Grâce',
 'Sorbonne',
 'Rennes',
 'Montparnasse',
 'Saint-Germain des Près',
 'Odéon',
 'Saint-Placide',
 'Notre Dame des Champs',
 '75006 Paris 6e',
 'Saint-Michel',
 'Hoche Friedland',
 'Elysée',
 'Monceau',
 "Triangle d'Or",
 'Europe',
 'Saint-Philippe du Roule',
 'Mairie',
 '75008 Paris 8e',
 'Clichy',
 'Grands Boulevards',
 'Lorette',
 'Opéra',
 'Trudaine',
 'Porte Saint-Martin',
 '75009 Paris 9e',
 'Saint-Vincent de Paul',
 "Château d'Eau",
 'Goncourt',
 'Canal Saint-Martin',
 'Louis Blanc',
 'Belleville',
 '75010 Paris 10e',
 'Nation',
 'Léon-Blum',
 '75011 Paris 11e',
 'Bastille',
 '

In [64]:
from geopy.geocoders import Nominatim
import time

# Créer un objet géocodeur Nominatim
geolocator = Nominatim(user_agent="my_geocoder")

# Fonction pour obtenir les coordonnées géographiques d'une adresse
def obtenir_coordonnees(tableau_zones):
    try:
        location = geolocator.geocode(tableau_zones, timeout=5)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Erreur pour {tableau_zones}: {e}")
        return None, None

# Extraire seulement les premières adresses uniques
list_quartiers_uniques_3 = tableau_zones[:10] 

# Créer un dictionnaire pour stocker les adresses et leurs coordonnées
coordonnees_dict = {}

for zone in list_quartiers_uniques_3:
    zone_paris = f"{zone}, Paris"  # Ajouter 'Paris' à chaque zone
    lat, lon = obtenir_coordonnees(zone_paris)  # Passer 'zone_paris' à la fonction
    coordonnees_dict[zone_paris] = (lat, lon)   # Utiliser 'zone_paris' comme clé
    
    # Pause pour éviter un blocage par le serveur
    time.sleep(1)

# Créer une liste pour stocker les informations des adresses
coordonnees2 = []

# Afficher et stocker les coordonnées pour les 3 premières adresses
print("\nCoordonnées des 3 premières adresses :")
for zone, coord in coordonnees_dict.items():
    print(f"Adresse: {zone}, Latitude: {coord[0]}, Longitude: {coord[1]}")
    coordonnees2.append({"adresse": zone, "latitude": coord[0], "longitude": coord[1]})  # Stocker les données

# Afficher le contenu de coordonnees2
print("\nContenu de coordonnees2 :")
for item in coordonnees2:
    print(item)


Coordonnées des 3 premières adresses :
Adresse: Châtelet, Paris, Latitude: 48.8597916, Longitude: 2.3466756
Adresse: Palais Royal, Paris, Latitude: 48.863584700000004, Longitude: 2.3362042200938715
Adresse: Saint-Germain, Paris, Latitude: 48.85655255, Longitude: 2.333331104782813
Adresse: Vendôme, Paris, Latitude: 48.86690005, Longitude: 2.3285489140926545
Adresse: Gaillon, Paris, Latitude: 48.869135150000005, Longitude: 2.332908770335507
Adresse: Saint-Thomas d'Aquin, Paris, Latitude: 48.85424375, Longitude: 2.3268829064838443
Adresse: 75001 Paris 1er, Paris, Latitude: 48.8572394, Longitude: 2.3471199
Adresse: Sentier, Paris, Latitude: 48.867518, Longitude: 2.3465656
Adresse: Montorgueil, Paris, Latitude: 48.8641678, Longitude: 2.3467038
Adresse: 75002 Paris 2e, Paris, Latitude: 48.8588897, Longitude: 2.3200410217200766

Contenu de coordonnees2 :
{'adresse': 'Châtelet, Paris', 'latitude': 48.8597916, 'longitude': 2.3466756}
{'adresse': 'Palais Royal, Paris', 'latitude': 48.8635847000

In [65]:
coordonnees2

[{'adresse': 'Châtelet, Paris',
  'latitude': 48.8597916,
  'longitude': 2.3466756},
 {'adresse': 'Palais Royal, Paris',
  'latitude': 48.863584700000004,
  'longitude': 2.3362042200938715},
 {'adresse': 'Saint-Germain, Paris',
  'latitude': 48.85655255,
  'longitude': 2.333331104782813},
 {'adresse': 'Vendôme, Paris',
  'latitude': 48.86690005,
  'longitude': 2.3285489140926545},
 {'adresse': 'Gaillon, Paris',
  'latitude': 48.869135150000005,
  'longitude': 2.332908770335507},
 {'adresse': "Saint-Thomas d'Aquin, Paris",
  'latitude': 48.85424375,
  'longitude': 2.3268829064838443},
 {'adresse': '75001 Paris 1er, Paris',
  'latitude': 48.8572394,
  'longitude': 2.3471199},
 {'adresse': 'Sentier, Paris', 'latitude': 48.867518, 'longitude': 2.3465656},
 {'adresse': 'Montorgueil, Paris',
  'latitude': 48.8641678,
  'longitude': 2.3467038},
 {'adresse': '75002 Paris 2e, Paris',
  'latitude': 48.8588897,
  'longitude': 2.3200410217200766}]

In [66]:
### CREATION DE LA CARTE ###

# Initialiser la carte centrée sur Paris
carte = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

# Ajouter des points sur la carte pour les adresses avec coordonnées
for lieu in coordonnees2 :
    if lieu["latitude"] is not None and lieu["longitude"] is not None:
        folium.Marker(
            location=[lieu["latitude"], lieu["longitude"]],
            popup=lieu["adresse"],  # Texte affiché lorsqu'on clique sur le point
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(carte)

# Sauvegarder la carte dans un fichier HTML
carte.save("carte_paris_points.html")
print("Carte créée : carte_paris_points.html")

Carte créée : carte_paris_points.html


In [67]:
carte